# Bayesian Optimization of an Attention Direction Classifier

In [10]:
# make sure you don't hog all the video memory
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from keras import backend as K
K.set_session(sess)
###################################

import GPy, GPyOpt
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Flatten, ELU, Permute
from keras.models import Model
from keras.models import load_model
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.initializers import Orthogonal as orth

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import scipy.io as sio


In [11]:
# EEG class
class EEG():
    def __init__(self,
                 SHUFFLE = True, BATCH_SIZE = 20, TEST_TRAIN = 0.25,
                 KER0=100, KER1=50, KER2=25, KER3=2, KER4=2,
                 N_KERNELS = (10,20,40,80,160),
                 POOL_SIZE = (5,2,5,2),
                 DO0 = 0.1, DO1 = 0.2, DO2 = 0.25, DO3 = 0.25, DO4 = 0.8, DO5 = 0.8,
                 DENSE_LEN = 10):
        self.__SHUFFLE = SHUFFLE
        self.__KER0 = KER0
        self.__KER1 = KER1
        self.__KER2 = KER2
        self.__KER3 = KER3
        self.__KER4 = KER4
        self.__BATCH_SIZE = BATCH_SIZE
        self.__TEST_TRAIN = TEST_TRAIN
        self.__N_KERNELS = N_KERNELS
        self.__POOL_SIZE = POOL_SIZE
        self.__DROPOUT = (DO0, DO1, DO2, DO3, DO4, DO5)
        self.__DENSE_LEN = DENSE_LEN
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.load_data()
        self.__model = self.CNN_model()
        
    # load dataset:
    def load_data(self):
        
#         mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/ksenia2_long_chunked.mat')
#         mat_contents = sio.loadmat('/home/amplifier/home/DATASETS/Merged456->2-30Hz,Env=1_TD1_94_.mat')
        mat_contents = sio.loadmat(
            '/home/amplifier/home/DATASETS/Merged123, DS2=80Hz, FIR=1-9Hz, centnorm=1, Env=1, TD, 143-202.mat')
        X = mat_contents['X']
        Y = mat_contents['Z']

        if X.shape[1]<X.shape[2]:
            X = np.transpose(X,[0,2,1])

        if Y.shape[1] > Y.shape[0]:
            Y = Y.T

#         print('Original data shape:', X.shape)
#         print('Original labels shape:', Y.shape)

        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=self.__TEST_TRAIN, shuffle=self.__SHUFFLE)
#         print('Original data type:', x_train.dtype)

        # convert to float64 for numerical stability:
        x_train = x_train.astype('float64')
        y_train = y_train.astype('float64')
        x_test = x_test.astype('float64')
        y_test = y_test.astype('float64')

        # one hot encode the labels:
        onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
        y_train = onehot_encoder.fit_transform(y_train)
        y_test = onehot_encoder.fit_transform(y_test)

        # convert to float16 to save space:
        x_train = x_train.astype('float16')
        y_train = y_train.astype('float16')
        x_test = x_test.astype('float16')
        y_test = y_test.astype('float16')
#         print('Normalized data type:', x_train.dtype)

        self.leng = X.shape[1] # if you work in the FD, this is the height of the sample time-frequency image, othewise EEG channels
        self.chan = X.shape[2] # if you work in the FD, this is the width of the sample time-frequency image, othewise time samples of EEG signal

        if len(X.shape)==3:
            streams = 1 # this is EEG channels if you work with frequency domain, in the TD streams = 1
        if len(X.shape)==4:
            streams = X.shape[3] # this is EEG channels if you work with frequency domain, in the TD streams = 1

#         print('test input shape', x_test.shape, "Nomralized MEAN:", np.mean(x_test), "min", np.min(x_test),"max", np.max(x_test))
#         print('train input shape', x_train.shape, "Nomralized MEAN:", np.mean(x_train), "min", np.min(x_train),"max", np.max(x_train))

#         print('test labels shape', y_test.shape, "Nomralized MEAN:", np.mean(y_test), "min", np.min(y_test),"max", np.max(y_test))
#         print('train labels shape', y_train.shape, "Nomralized MEAN:", np.mean(y_train), "min", np.min(y_train),"max", np.max(y_train))

        return x_train, x_test, y_train, y_test
    
    def activation(self):
        return ELU(alpha=1.0) # Activation('relu') #
    
    # define model
    def CNN_model(self):
        N_KERNELS = self.__N_KERNELS
        POOL_SIZE = self.__POOL_SIZE
        KER0 = self.__KER0,
        KER1 = self.__KER1,
        KER2= self.__KER2,
        KER3 = self.__KER3,
        KER4 = self.__KER4,
        DROPOUT = self.__DROPOUT
        DENSE_LEN = self.__DENSE_LEN
        
        input_img = Input(shape=(self.leng, self.chan))  # adapt this if using `channels_first` image
        input_norm = BatchNormalization()(input_img)

        e2 = Conv1D(N_KERNELS[0], KER0, padding='same',
                                        kernel_initializer = orth(gain=1.0, seed=None))(input_norm)
        e2act = self.activation()(e2)
        e2p = MaxPooling1D(pool_size=POOL_SIZE[0], strides=None, padding='same')(e2act)
        e2b = BatchNormalization()(e2p)
        e2a = Dropout(DROPOUT[0])(e2b)


        e3 = Conv1D(N_KERNELS[1], KER1, padding='same',
                                        kernel_initializer = orth(gain=1.0, seed=None))(e2a)
        e3act = self.activation()(e3)
        e4 = MaxPooling1D(pool_size=POOL_SIZE[1], strides=None, padding='same')(e3act)
        e4b = BatchNormalization()(e4)
        e4a = Dropout(DROPOUT[1])(e4b)


        e5 = Conv1D(N_KERNELS[2], KER2, padding='same',
                                        kernel_initializer = orth(gain=1.0, seed=None))(e4a)
        e5act = self.activation()(e5)
        e6 = MaxPooling1D(pool_size=POOL_SIZE[2], strides=None, padding='same')(e5act)
        e6b = BatchNormalization()(e6)
        e6a = Dropout(DROPOUT[2])(e6b)


        e7 = Conv1D(N_KERNELS[3], KER3, padding='same',
                                        kernel_initializer = orth(gain=1.0, seed=None))(e6a)
        e7act = self.activation()(e7)
        e8 = MaxPooling1D(pool_size=POOL_SIZE[3], strides=None, padding='same')(e7act)
        e8b = BatchNormalization()(e8)
        e9a = Dropout(DROPOUT[3])(e8b)


        e10 = Conv1D(N_KERNELS[4], KER4, padding='same',
                                        kernel_initializer = orth(gain=1.0, seed=None))(e9a)
        e10act = self.activation()(e10)
        e10b = BatchNormalization()(e10act)
        e10a = Dropout(DROPOUT[4])(e10b)


        e11 = Flatten()(e10a)

        e12 = Dense(DENSE_LEN, activation='sigmoid',
                                         kernel_initializer = orth(gain=1.0, seed=None))(e11)
        e12b = BatchNormalization()(e12)
        e12a = Dropout(DROPOUT[5])(e12b)

        e13 = Dense(2, activation='softmax')(e12a)
        
        model = Model(input_img, e13)
        model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])
#         model.summary()
        return model
    
    # fit the model
    def CNN_fit(self):
        early_stopping = EarlyStopping(monitor='val_loss', patience=50, mode='min')
        
        checkpointer = ModelCheckpoint(
            filepath='/home/amplifier/home/NEW_DL/weights/EEGnet_weights_TD_1D.h5',
            verbose=0,
            monitor='val_acc',
            save_best_only=True)
        
        train_history = self.__model.fit(self.__x_train, self.__y_train,
                epochs=1000,
                batch_size=self.__BATCH_SIZE,
                verbose=0,
                shuffle=True,
                validation_data=(self.__x_test, self.__y_test),
                callbacks=[checkpointer, early_stopping])
    
    # evaluate the model
    def CNN_evaluate(self):
        self.CNN_fit()
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, 
                                           batch_size=self.__BATCH_SIZE, verbose=0)
        return evaluation

In [12]:
# function to define model, read in data:
def run_CNN(SHUFFLE = True, BATCH_SIZE = 20, TEST_TRAIN = 0.25,
            
            KER0=100,
            KER1=50,
            KER2=25,
            KER3=2,
            KER4=2,
            N_KERNELS = (10,20,40,80,160),
            POOL_SIZE = (5,2,5,2),
            DO0 = 0.1, DO1 = 0.2, DO2 = 0.25, DO3 = 0.25, DO4 = 0.8, DO5 = 0.8,
            DENSE_LEN = 10):
    
    _eeg = EEG(SHUFFLE = SHUFFLE, BATCH_SIZE = BATCH_SIZE, TEST_TRAIN = TEST_TRAIN,
               KER0 = KER0, KER1 = KER1, KER2 = KER2, KER3 = KER3, KER4 = KER4,
               N_KERNELS = N_KERNELS,
               POOL_SIZE = POOL_SIZE, 
               DO0 = DO0, DO1 = DO1, DO2 = DO2, DO3 = DO3, DO4 = DO4, DO5 = DO5,
               DENSE_LEN = DENSE_LEN)
    CNN_evaluation = _eeg.CNN_evaluate()
    return CNN_evaluation

In [13]:
# run_CNN()

In [14]:
# bounds for hyper-parameters in CNN model
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'KER0',             'type': 'discrete',    'domain': (30, 50, 70, 100)},
          {'name': 'KER1',             'type': 'discrete',    'domain': (30, 50, 70)},
          {'name': 'KER2',             'type': 'discrete',    'domain': (10, 20, 30, 40)},
          {'name': 'KER3',             'type': 'discrete',    'domain': (4, 6, 8)},
          {'name': 'DO3',              'type': 'discrete',    'domain': (0.4, 0.7, 0.9)},
          {'name': 'DO4',              'type': 'discrete',    'domain': (0.4, 0.7, 0.9)},
          {'name': 'DO5',              'type': 'discrete',    'domain': (0.5, 0.7, 0.9)},
          {'name': 'DENSE_LEN',        'type': 'discrete',    'domain': (10, 20)}]

In [15]:
# function to optimize the CNN model
def f(x):
    print(x)
    evaluation = run_CNN(
        KER0 = int(x[:,0]), 
        KER1 = int(x[:,1]),
        KER2 = int(x[:,2]), 
        KER3 = int(x[:,3]), 
        DO3 = int(x[:,4]),
        DO4 = int(x[:,5]),
        DO5 = int(x[:,6]),
        DENSE_LEN = int(x[:,7]))
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    return evaluation[0]

In [16]:
# run Bayesian optimization of the hyperparameters
opt_hparams = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)

[[ 100.    30.    40.     4.     0.7    0.4    0.5   20. ]]
LOSS:	1.9087007309483215 	 ACCURACY:	0.5398230046297596
[[ 50.   50.   30.    8.    0.9   0.7   0.9  20. ]]
LOSS:	1.803819335667433 	 ACCURACY:	0.5752212510699719
[[ 50.   50.   20.    4.    0.7   0.4   0.9  20. ]]
LOSS:	1.8839498695019072 	 ACCURACY:	0.5663716782510808
[[ 30.   70.   20.    4.    0.4   0.9   0.7  20. ]]
LOSS:	2.2141897150900514 	 ACCURACY:	0.5044247837720719
[[ 100.    30.    20.     8.     0.7    0.9    0.7   10. ]]
LOSS:	1.8676535998825479 	 ACCURACY:	0.5044247784973246


In [17]:
opt_hparams.run_optimization(max_iter=50)

[[ 50.   50.   30.    8.    0.9   0.7   0.9  20. ]]
LOSS:	2.4080165339782176 	 ACCURACY:	0.46017699431529085
[[ 100.    30.    20.     8.     0.7    0.9    0.9   10. ]]
LOSS:	1.9100385186946498 	 ACCURACY:	0.5398230204540017
[[ 100.    30.    20.     8.     0.4    0.9    0.9   10. ]]
LOSS:	1.717380706187898 	 ACCURACY:	0.5486725800860245
[[ 100.    30.    20.     8.     0.7    0.9    0.9   10. ]]
LOSS:	1.8664990515835518 	 ACCURACY:	0.5132743441953068
[[ 100.    30.    20.     8.     0.7    0.7    0.9   10. ]]
LOSS:	1.9729630451286788 	 ACCURACY:	0.5132743460414684
[[ 100.    30.    20.     8.     0.7    0.4    0.7   10. ]]
LOSS:	2.3170954716944063 	 ACCURACY:	0.46902655130994003
[[ 100.    30.    20.     8.     0.4    0.9    0.7   10. ]]
LOSS:	1.8183828792740813 	 ACCURACY:	0.566371682734616
[[ 100.    30.    20.     8.     0.9    0.9    0.9   10. ]]
LOSS:	1.693705149456463 	 ACCURACY:	0.5840707967239144
[[ 100.    30.    20.     8.     0.7    0.9    0.9   10. ]]
LOSS:	2.0727604152881

In [18]:
# print optimized mnist model
print("""
Optimized Parameters:
\t{0}:\t\t{1}
\t{2}:\t\t{3}
\t{4}:\t\t{5}
\t{6}:\t\t{7}
\t{8}:\t\t{9}
\t{10}:\t\t{11}
\t{12}:\t\t{13}
\t{14}:\t{15}
""".format(bounds[0]["name"],opt_hparams.x_opt[0],
           bounds[1]["name"],opt_hparams.x_opt[1],
           bounds[2]["name"],opt_hparams.x_opt[2],
           bounds[3]["name"],opt_hparams.x_opt[3],
           bounds[4]["name"],opt_hparams.x_opt[4],
          bounds[5]["name"],opt_hparams.x_opt[5],
          bounds[6]["name"],opt_hparams.x_opt[6],
          bounds[7]["name"],opt_hparams.x_opt[7]))
print("optimized loss: {0}".format(opt_hparams.fx_opt))


Optimized Parameters:
	KER0:		100.0
	KER1:		30.0
	KER2:		20.0
	KER3:		8.0
	DO3:		0.9
	DO4:		0.9
	DO5:		0.9
	DENSE_LEN:	10.0

optimized loss: [ 1.49330802]
